In [1]:
import pandas as pd


In [2]:
GSS = pd.read_pickle('data0725/GSS_top30.pk')
HMA = pd.read_pickle('data0725/HMA_top30.pk')
OLI = pd.read_pickle('data0725/OLI_top30.pk')

In [14]:
HMA.head(1)

,ITEM_ID,STOCK_ID,ITEM_NUM,GOODS_NAME,NORMAL_PRICE_SITE,BRAND_NAME,CATE_NAMES,BRAND,SITE_NO,COLLECT_URL,SELL_AMOUNT
24414,6665382,13062930,2059596366,라네즈 워터슬리핑 마스크 70ML (구매금액별 증정 제외),28000,라네즈,스킨케어>마스크/팩,AP,3,http://www.hyundaihmall.com/front/pda/itemPtc....,4443.0


### PreProcessing - removal

In [ ]:
# GSS[GSS.BRAND=='LVMH'].sort_values(by=['SELL_AMOUNT'],ascending=False)
# GSS[GSS.BRAND=='init'].sort_values(by=['SELL_AMOUNT'],ascending=False)
GSS[(GSS.ITEM_ID==8866616)&(GSS.BRAND=='LVMH')].sort_values(by=['SELL_AMOUNT'],ascending=False)

In [16]:
#REMOVAL                                                            
removal = pd.DataFrame([])
removal['ITEM_ID'] = [8866616,1907459,8427631,447103,1909086,11412866]
removal['BRAND'] = ['LVMH','init','init','init','init','init']
removal

,ITEM_ID,BRAND
0,8866616,LVMH
1,1907459,init
2,8427631,init
3,447103,init
4,1909086,init
5,11412866,init


In [28]:
for df in [GSS,HMA,OLI]:
    print(df.shape)
    for index, data in removal.iterrows():
        df=df[~((df.ITEM_ID ==data['ITEM_ID'])&(df.BRAND ==data['BRAND']))]
        print(index,df[~((df.ITEM_ID ==data['ITEM_ID'])&(df.BRAND ==data['BRAND']))].shape)

(500, 11)
0 (499, 11)
1 (498, 11)
2 (497, 11)
3 (496, 11)
4 (495, 11)
5 (494, 11)
(500, 11)
0 (500, 11)
1 (500, 11)
2 (500, 11)
3 (500, 11)
4 (500, 11)
5 (500, 11)
(485, 11)
0 (485, 11)
1 (485, 11)
2 (485, 11)
3 (485, 11)
4 (485, 11)
5 (485, 11)


In [31]:
for index, data in removal.iterrows():
    GSS=GSS[~((GSS.ITEM_ID ==data['ITEM_ID'])&(GSS.BRAND ==data['BRAND']))]
    print(index,GSS[~((GSS.ITEM_ID ==data['ITEM_ID'])&(GSS.BRAND ==data['BRAND']))].shape)
GSS.shape

0 (499, 11)
1 (498, 11)
2 (497, 11)
3 (496, 11)
4 (495, 11)
5 (494, 11)


(494, 11)

### PreProcessing - ADD COLUMNS
* SITENAME 쇼핑몰
* BRAND 제조사
* CATE1, CATE2 카테고리 분리
* AMT_SALES 매출액

In [75]:
def map_site_noToName(siteNo):
    if siteNo == 1:
        return 'GSSHOP'
    elif siteNo == 3:
        return 'HMALL'
    elif siteNo == 4 :
        return 'OLIVE'
def map_Brand(br):
    if br == 'AP':
        return '아모레퍼시픽'
    elif br == 'LG':
        return 'LG화장품'
    elif br == 'LOREAL':
        return '로레알'
    elif br == 'init' :
        return '4개사 이외'
    else :
        return br


In [99]:
def refine_df(df):
    df=df.assign(SITENAME = df.apply(lambda x : map_site_noToName(x['SITE_NO']),axis=1))
    df=df.assign(BRAND = df.apply(lambda x : map_Brand(x['BRAND']),axis=1))
    df = df.assign(CATE1=df.apply(lambda x : x.CATE_NAMES.split('>')[0],axis=1))
    df = df.assign(CATE2=df.apply(lambda x : x.CATE_NAMES.split('>')[1],axis=1))
    df=df.assign(AMT_SALES=df.apply(lambda x : x['NORMAL_PRICE_SITE']*x['SELL_AMOUNT'],axis=1))
    df=df.reset_index()
    df = df.sort_values(by=['AMT_SALES'],ascending =False)[:100]
    df = df[['GOODS_NAME','NORMAL_PRICE_SITE','BRAND_NAME',\
                 'CATE1','CATE2','BRAND','SITENAME','COLLECT_URL','SELL_AMOUNT','AMT_SALES']]
    df.columns =['상품명','정상가','브랜드',\
             '카테고리1','카테고리2','제조사','쇼핑몰','상품URL','판매량','매출액']   
    
    return df

### 전체- 사이트별(브랜드상관없이)

In [100]:
df_list = []

writer = pd.ExcelWriter('data0725/total_Bysite.xlsx')
for  key, dfs in {'GSS':GSS,'HMA':HMA,'OLI':OLI}.items():
    refine_df(dfs).to_excel(writer,key+'_매출액순')
    df_list.append(refine_df(dfs))
# df_list[0].head()
writer.save()

In [101]:
df_list[0].head()

,상품명,정상가,브랜드,카테고리1,카테고리2,제조사,쇼핑몰,상품URL,판매량,매출액
110,[LG전자][18만원할인가] 프라엘 심화세트 리프트케어+LED마스크,1016000,LG전자,스킨케어,마스크/팩,LG화장품,GSSHOP,http://www.gsshop.com/prd/prd.gs?prdid=29145998,1002.0,1.018032e+09
465,실큰 플래시앤고 IPL 제모기+추가 카트리지 (총 40만 5천회),299000,실큰,바디/헤어,툴/소품,4개사 이외,GSSHOP,http://www.gsshop.com/prd/prd.gs?prdid=31241792,2877.0,8.602230e+08
424,사이오스 골드라벨 헤어컬러 12개+샴푸1+컨디1,79800,사이오스,바디/헤어,헤어케어,4개사 이외,GSSHOP,http://www.gsshop.com/prd/prd.gs?prdid=26178265,5985.0,4.776030e+08
405,키코 KIKO 오일프리 메트 Mousse 파운데이션,30500,미분류,메이크업,베이스,4개사 이외,GSSHOP,http://www.gsshop.com/prd/prd.gs?prdid=29099905,14000.0,4.270000e+08
302,[프레쉬] 향수 100ml (11종 택1),131750,프레쉬,향수,남녀공용향수,LVMH,GSSHOP,http://www.gsshop.com/prd/prd.gs?prdid=28964820,2987.0,3.935372e+08


### 브랜드- 사이트별
* 아모레
* 로레알
* LG
* LVMH

In [102]:
GSS.columns
GSS.head(1)

,ITEM_ID,STOCK_ID,ITEM_NUM,GOODS_NAME,NORMAL_PRICE_SITE,BRAND_NAME,CATE_NAMES,BRAND,SITE_NO,COLLECT_URL,SELL_AMOUNT
41049,7541381,29375979001,29375979,이니스프리 마이 파운데이션 (50종 택1),20000,이니스프리,메이크업>베이스,AP,1,http://www.gsshop.com/prd/prd.gs?prdid=29375979,4886.0


In [103]:
GSS.BRAND.value_counts()

LG        100
LOREAL    100
AP        100
LVMH       99
init       95
Name: BRAND, dtype: int64

In [104]:
for brand in ['AP','LG','LOREAL','LVMH']: 
    writer = pd.ExcelWriter('data0725/'+brand+'_Bysite.xlsx')
    for  key, dfs in {'GSS':GSS[GSS.BRAND==brand],'HMA':HMA[HMA.BRAND==brand],'OLI':OLI[OLI.BRAND==brand]}.items():
        refine_df(dfs).to_excel(writer,key+'_매출액순')
    writer.save()